In [24]:
import pandas as pd
import os

def process_all_condition_files(folder_path, groups_csv_filename, output_csv_filename):
    # קריאת טבלת קבוצות
    groups_df = pd.read_csv(os.path.join(folder_path, groups_csv_filename))
    groups_df.columns = ["Rat", "Group"]

    # קבצי CSV בתיקייה, למעט קובץ הקבוצות
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f != groups_csv_filename]

    master_df = None

    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        condition = os.path.splitext(file_name)[0]
        print(f"מעבד קובץ: {file_name}")

        df = pd.read_csv(file_path, header=None, names=["ID", "Score"])

        # סינון שורות עם מבנה ID תקני (2 נקודות בדיוק)
        df = df[df["ID"].str.count(r"\.") == 2].copy()

        # פיצול ID לשלושה חלקים
        split_cols = df["ID"].str.split(".", expand=True)
        if split_cols.shape[1] != 3:
            print(f"שורה בעייתית בקובץ {file_name} - דילוג")
            continue

        df[["Rat", "Slide", "Index"]] = split_cols
        df = df.dropna(subset=["Rat", "Slide", "Index"])
        df["Rat"] = pd.to_numeric(df["Rat"], errors="coerce")
        df = df.dropna(subset=["Rat"])
        df["Rat"] = df["Rat"].astype(int)
        df["Score"] = pd.to_numeric(df["Score"], errors="coerce")

        df = df.sort_values(by=["Rat", "Slide", "Index"])
        df["Measurement"] = df.groupby("Rat").cumcount() + 1

        pivot = df.pivot_table(index="Rat", columns="Measurement", values="Score", aggfunc="first")
        pivot.columns = [f"{condition}{i}" for i in pivot.columns]
        pivot.reset_index(inplace=True)
        pivot[f"{condition}.mean()"] = pivot.loc[:, pivot.columns.str.startswith(condition)].mean(axis=1)

        if master_df is None:
            master_df = pivot
        else:
            master_df = pd.merge(master_df, pivot, on="Rat", how="outer")

    # חיבור לקבוצות
    final_df = pd.merge(groups_df, master_df, on="Rat", how="right")
    final_df.rename(columns={"Rat": "מספר חולדה", "Group": "קבוצה"}, inplace=True)

    # שמירה
    output_path = os.path.join(folder_path, output_csv_filename)
    final_df.to_csv(output_path, index=False)
    return final_df






In [25]:

# הרצה עם קבצים שהועלו
folder_path = "/Users/djourno/PyCharm/MDMA"
groups_csv_filename = "groups.csv"
output_csv_filename = "Brains.csv"

final_df = process_all_condition_files(folder_path, groups_csv_filename, output_csv_filename)


מעבד קובץ: BDNF_CA3.csv
מעבד קובץ: BDNF_CA1.csv
מעבד קובץ: TPH2.csv
מעבד קובץ: Oxy.csv
מעבד קובץ: BDNF_RAPHE.csv
מעבד קובץ: GR.csv
מעבד קובץ: DG_GOLGI.csv
מעבד קובץ: BDNF_PVN.csv
